In [1]:
import pandas as pd
from tqdm import trange

import sys
from pathlib import Path
import os

sys.path.append(str(Path.cwd().parent))
sys.path.append(os.path.join(os.getcwd(),'../scripts'))

from scripts.llm_anotation import anotate_paragraphs
import json

df = pd.read_csv('../data/final_paragraphs/sampled_paragraphs.csv')

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
df['tokens'] = df['text'].apply(lambda x: [[i,x] for x,i in enumerate(tokenizer.tokenize(x))])

/home/mkdsp/Desktop/HarryPotter/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def convert_to_list(df):
    result = []
    for _, row in df.iterrows():
        result.append({
            'id': row['index'],
            'text': row['text'],
            'tokens': row['tokens']
        })
    return result

In [4]:
def tokens_checker(data_point: dict):
    def clean_string(s):
        return s.replace('#', '').replace(' ', '')

    valid = True

    tokens = data_point.get('tokens', [])

    for entity in data_point.get('entities', []):
        start = entity.get('token_start', -1)
        end = entity.get('token_end', -1)

        # Provera granica
        if 0 <= start <= end < len(tokens):
            entity_token_text = ''.join([x[0] for x in tokens[start : end + 1]])
        else:
            entity_token_text = ''
        
        entity_text = entity.get('text', '')
        token_validity = clean_string(entity_text) == clean_string(entity_token_text)
        entity['token_validity'] = token_validity
        valid &= token_validity

    data_point['token_validity'] = valid
    return data_point

In [5]:
# # OVO JE ZA TESTIRANJE
# group = convert_to_list(df[df['index'].isin([14396, 17212, 4346])])
# data = anotate_paragraphs(group)
# step = 3

# for j in range(len(data['data_points'])):
#     row = df[df['index'] == data['data_points'][j]['id']].iloc[0]
#     data['data_points'][j]['text'] = row['text']
#     data['data_points'][j]['tokens'] = row['tokens']

#     data['data_points'][j] = tokens_checker(data['data_points'][j])

# with open('output_pokusaj2.json', 'w') as f:
#     json.dump(data, f, indent=1)


In [6]:
step = 3
for i in trange(0, len(df), step):
    group = df.iloc[i:i+step]

    try:
        data = anotate_paragraphs(convert_to_list(group))

        for j in range(min(step, len(data['data_points']), len(group))):
            data['data_points'][j]['text'] = group.iloc[j]['text']
            data['data_points'][j]['tokens'] = group.iloc[j]['tokens']
            data['data_points'][j] = tokens_checker(data['data_points'][j])
        file = os.path.join("../anotations/json_files", f'output{int(i/step)}.json')

        with open(file, 'w') as f:
            json.dump(data, f, indent=1)

    
    except Exception as e:
        print(f"Došlo je do greške pri obradi pasusa {i}-{i+step}: {e}")
        raise
    
    
    

100%|██████████| 1659/1659 [3:28:06<00:00,  7.53s/it]  
